# Turnbell Model

In [1]:
import numpy as np

In [4]:
import os

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
from matplotlib.backends.backend_pdf import PdfPages

In [5]:
%run ../models/drift/wagner.py

In [6]:
%run ../models/iceberg/iceberg.py

In [18]:
%run ../models/metocean/metocean.py

In [12]:
%run ../plotting/plot.py

In [13]:
chosen_track_ind = 0
iip_berg_df = get_berg_df(chosen_track_ind)
iip_berg_df

,index,ICE-YEAR,BERG_NUMBER,DATE,TIME,LATITUDE,LONGITUDE,METHOD,SIZE,SHAPE,SOURCE,DATETIME,t2000
0,6319,2015,21511,6/16/2015,1442,48.16,-50.01,R/V,LG,TAB,GTJZ,2015-06-16 14:42:00,135494.70
1,6320,2015,21511,6/17/2015,1300,48.09,-50.02,R/V,LG,GEN,GMRS,2015-06-17 13:00:00,135517.00
2,6321,2015,21511,6/18/2015,1249,48.08,-49.76,VIS,LG,TAB,GTJZ,2015-06-18 12:49:00,135540.82


In [14]:
iip_berg = Iceberg(iip_berg_df['BERG_NUMBER'].loc[0],
                       iip_berg_df['DATETIME'].loc[:].tolist(),
                       iip_berg_df['t2000'].loc[:].tolist(),
                       iip_berg_df['LATITUDE'].loc[:].tolist(),
                       iip_berg_df['LONGITUDE'].loc[:].tolist(),
                       iip_berg_df['SIZE'].loc[0])

In [19]:
atm_data = ECMWF_Atm(min(iip_berg.lons), max(iip_berg.lons),
                     min(iip_berg.lats), max(iip_berg.lats),
                     iip_berg.times[0], iip_berg.times[-1])

In [20]:
ocean_data = ECMWF_Ocean(min(iip_berg.lons), max(iip_berg.lons),
                         min(iip_berg.lats), max(iip_berg.lats),
                         iip_berg.times[0], iip_berg.times[-1])

In [22]:
AU = atm_data.wind_u_interp
AV = atm_data.wind_v_interp  
WU = ocean_data.water_u_interp 
WV = ocean_data.water_v_interp

In [25]:
berg_size = 'LG'
mod_berg = Iceberg(iip_berg_df['BERG_NUMBER'].loc[0],
                   [iip_berg_df['DATETIME'].loc[0]],
                   [iip_berg_df['t2000'].loc[0]],
                   [iip_berg_df['LATITUDE'].loc[0]],
                   [iip_berg_df['LONGITUDE'].loc[0]],
                   berg_size)
mod_berg.length = [mod_berg.length]
mod_berg.width = [mod_berg.width]
mod_berg.height = [mod_berg.height]

In [31]:
x0, y0 = mod_berg.lons[0], mod_berg.lats[0]
vx0 = 0
vy0 = 0
t0 = iip_berg.t2000[0]
tf = iip_berg.t2000[-1]
l0, w0, h0 = mod_berg.length[0], mod_berg.width[0], mod_berg.height[0]

In [43]:
vx, vy, x, y = turnbell_drift(t0, vx0, vy0, x0, y0, l0, w0, h0, AU, AV, WU, WV)

In [44]:
x

-50.009999999999998

In [45]:
y

48.159999999999997

In [41]:
def turnbell_drift(t, vx, vy, x, y, l, w, h, AU, AV, WU, WV):
    
    dt = 60.0
        
    Vx = vx
    Vy = vy
        
    # Air drag
    Vax = 0*60.0*np.cos(1*np.pi*t/86400) # nominal value
    Vay = 0
    
    # calculate air density
    # TODO: effect of temperature and humidity
    rhoa = 1.225 # density of air (kg/m^3)
    
    Ca = 0.1  # air drag coefficient
    As = 72 * 22  # sail area (maximum width (m) * sail height (m))
    
    # TODO: add skin drag
    Fax = (0.5 * rhoa * Ca * As) * abs(Vax - Vx) * (Vax - Vx)
    Fay = (0.5 * rhoa * Ca * As) * abs(Vay - Vy) * (Vay - Vy)
    
    
    # Water drag  
    Vcx = 0
    Vcy = 0
    
    rhow = 1027.5  # density of water (kg/m^3)
    Cw = 2.5    # water drag coefficient
    Ak = 7*110    # keel area (width (m) * keel depth (m))
    
    Cdw = 5.0e-4
    
    # TODO: extend to n-layer keel model
    # TODO: add skin drag
    Fwx = (0.5 * rhow * Cw * Ak *abs(Vcx - Vx)*(Vcx-Vx)) +(0)
    Fwy = (0.5 * rhow * Cw * Ak *abs(Vcy - Vy)*(Vcy-Vy)) +(0)
    
    
    # Coriolis force
    om = 7.2921e-5  # rotation rate of earth in rad/s
    f = 2*om*np.sin(np.deg2rad(y))  # Coriolis parameter (y is latitude in degrees)
    rhoi = 850
    berg_mass = l*w*h*rhoi
    Fcx = + f * Vy * berg_mass
    Fcy = - f * Vx * berg_mass
    
    
    # Water pressure gradient
    # Mean water current down to the iceberg keel
    Vwmx = 0
    Vwmy = 0
    
    # acceleration (time-derivative) of Vmw
    Amwx = 0
    Amwy = 0
        
    Fwpx = berg_mass*(Amwx + f*Vwmx)
    Fwpy = berg_mass*(Amwy - f*Vwmy)
    
    
    # Iceberg acceleration
    
    # added mass
    Ma = 0.5*berg_mass
                
    ax = (Fax + Fcx + Fwx + Fwpx) / (berg_mass + Ma)
    ay = (Fay + Fcy + Fwy + Fwpy) / (berg_mass + Ma)
    
    vx += dt*ax
    vy += dt*ay
    
    earth_radius = 6378*1e3
    y_new = y + dt*vy*(180/(np.pi*earth_radius))
    x_new = x + dt*vx/(np.cos((((y + y_new)/2)*np.pi)/180))*(180/(np.pi*earth_radius))
    
    return vx, vy, x_new, y_new
    

In [42]:
def solve(berg, dt=60.0):
    """
    Solve the equations of motion with
        dt time step in seconds
        
        berg is an Iceberg
    """
    tmax_hours = 72
    tmax = tmax_hours*60*60 # hours to seconds
    
    # total number of timesteps
    N = round(tmax/dt)

    # allocate memory for arrays
    x = np.zeros(N)
    y = np.zeros(N)
    vx = np.zeros(N)
    vy = np.zeros(N)
    ax = np.zeros(N)
    ay = np.zeros(N)
    t = np.zeros(N)

    # initial values
    x[0] = 0
    y[0] = 0
    vx[0] = 2
    vy[0] = 0
    ax[0], ay[0] = turnbell_drift(berg, vx[0], vy[0], t[0])
    t[0] = 0

    # integrate numerically
    for i in range(N-1):
        
        t[i+1] = t[i] + dt
            
        if i < 1:
            # explicit Euler forward scheme
            vx[i+1] = vx[i] + dt*ax[i]
            vy[i+1] = vy[i] + dt*ay[i]  
            ax[i+1], ay[i+1] = turnbell_drift(t[i+1], vx[i+1], vy[i+1])
            
        elif i < 3:
            # second order Adams Bashforth
            vx[i+1] = vx[i] + dt*(1.5*ax[i]-0.5*ax[i-1])
            vy[i+1] = vy[i] + dt*(1.5*ay[i]-0.5*ay[i-1]) 
            ax[i+1], ay[i+1] = turnbell_drift(t[i+1], vx[i+1], vy[i+1])
            
        else:
            # fourth order Adams Bashforth, predictor-corrector
            vx[i+1] = vx[i] + dt/24*(55*ax[i]-59*ax[i-1]+37*ax[i-2]-9*ax[i-3])
            vy[i+1] = vy[i] + dt/24*(55*ay[i]-59*ay[i-1]+37*ay[i-2]-9*ay[i-3])    
            ax[i+1], ay[i+1] = turnbell_drift(t[i+1], vx[i+1], vy[i+1])
            
            vx[i+1] = vx[i] + dt/24*(9*ax[i+1]+19*ax[i]-5*ax[i-1]+ax[i-2])
            vy[i+1] = vy[i] + dt/24*(9*ay[i+1]+19*ay[i]-5*ay[i-1]+ay[i-2]) 
            ax[i+1], ay[i+1] = turnbell_drift(t[i+1], vx[i+1], vy[i+1])
            
        x[i+1] = x[i] + dt*vx[i+1]
        y[i+1] = y[i] + dt*vy[i+1]
  
    
    return x, y, t